# GraphRAG

## Import packages

In [1]:
! pip install nltk numpy pandas unidecode scikit-learn tqdm
! pip install langchain langchain-core langchain-community langchain_experimental langchain-chroma langchain_mistralai langgraph

In [2]:
import os
import re
import nltk
import string
import numpy as np
import pandas as pd
from unidecode import unidecode
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from pathlib import Path
import pickle
from rouge_score import rouge_scorer

from langchain_community.document_loaders import PDFMinerLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain.embeddings.cache import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain_community.llms import Ollama
from langgraph.graph import END, StateGraph
from langchain_core.output_parsers import PydanticOutputParser
from langchain.output_parsers import RetryOutputParser
from typing import Literal
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnableLambda, RunnableParallel
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

## Disable warnings

In [3]:
import warnings
warnings.filterwarnings('ignore')

## Setup metrics

### Download NLTK dictionaries

These dictionaries are needed for further text preprocessing.

In [4]:
dict_ids = [
  'punkt_tab',
  'punkt',
  'stopwords',
  'wordnet',
]

for dict_id in dict_ids:
  nltk.download(dict_id, quiet=True)

### Text preprocessing

Define a function for text preprocessing, which is an important step before calculating any metrics. This preprocessing function will help in cleaning the text data, making it ready for further analysis. The preprocessing involves several steps:
1. Lowercasing
2. Stopwords removal
3. Lemmatization
4. Remove accents from characters

In [5]:
lemmatizer = nltk.stem.WordNetLemmatizer()

def preprocess(corpus: str) -> str:
  corpus = corpus.lower()
  stopset = nltk.corpus.stopwords.words('english') + nltk.corpus.stopwords.words('russian') + list(string.punctuation)
  tokens = nltk.word_tokenize(corpus)
  tokens = [t for t in tokens if t not in stopset]
  tokens = [lemmatizer.lemmatize(t) for t in tokens]
  corpus = ' '.join(tokens)
  corpus = unidecode(corpus)
  return corpus

### Embedding Initialization

Here we are initializing the Llama 3 embeddings model. The `OllamaEmbeddings` class is a component of the Ollama library, a set of pre-trained language models. This model is capable of embedding corpora of any length into a 4096-dimensional vector.

The use of `OllamaEmbeddings` requires the installation of a local Ollama server, which can be found at https://ollama.com.

In [6]:
embeddings = OllamaEmbeddings(model='llama3.1')
store = LocalFileStore("./.embeddings_cache")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
  embeddings,
  store,
  namespace=embeddings.model,
)

### Average embeddings cosine similarity metric

This function calculates the average cosine similarity between expected answers and LLM predicted answers using their respective embeddings. Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them:

$$
K(a, b) = \frac{\sum \limits_{i=1}^n a_i b_i}{\sqrt{\sum \limits_{i=1}^n a_i^2} \cdot \sqrt{\sum \limits_{i=1}^n b_i^2}}
$$

In [7]:
def embeddings_cosine_sim_metric(expected_answers: list[str], predicted_answers: list[str]) -> float:
  results = []

  for expected_answer, predicted_answer in zip(expected_answers, predicted_answers):
    expected_answer = preprocess(expected_answer)
    predicted_answer = preprocess(predicted_answer)

    expected_embedding = np.array(cached_embeddings.embed_query(expected_answer))
    predicted_embedding = np.array(cached_embeddings.embed_query(predicted_answer))

    sim = cosine_similarity(
      expected_embedding.reshape(1, -1),
      predicted_embedding.reshape(1, -1),
    )[0][0]

    results.append(sim)

  return np.mean(results)

In [8]:
smoothie_f = nltk.translate.bleu_score.SmoothingFunction().method4

def bleu_metric(expected_answers, predicted_answers):
  scores = []

  for expected_answer, predicted_answer in zip(expected_answers, predicted_answers):
    expected_answer = preprocess(expected_answer)
    predicted_answer = preprocess(predicted_answer)

    predicted_tokens = nltk.word_tokenize(predicted_answer)
    expected_tokens = [nltk.word_tokenize(expected_answer)]

    score = nltk.translate.bleu_score.sentence_bleu(
      expected_tokens,
      predicted_tokens,
      smoothing_function=smoothie_f,
    )

    scores.append(score)

  return np.mean(scores)

In [9]:
rogue_1_scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

def rogue_1_metric(expected_answers, predicted_answers):
  scores = []

  for expected_answer, predicted_answer in zip(expected_answers, predicted_answers):
    expected_answer = preprocess(expected_answer)
    predicted_answer = preprocess(predicted_answer)

    result = rogue_1_scorer.score(expected_answer, predicted_answer)

    scores.append(result['rouge1'])

  return np.mean(scores)

In [10]:
rogue_l_scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def rogue_l_metric(expected_answers, predicted_answers):
  scores = []

  for expected_answer, predicted_answer in zip(expected_answers, predicted_answers):
    expected_answer = preprocess(expected_answer)
    predicted_answer = preprocess(predicted_answer)

    result = rogue_l_scorer.score(expected_answer, predicted_answer)

    scores.append(result['rougeL'])

  return np.mean(scores)

## Load documents

In [11]:
docs_dir = Path('./docs')
docs_cache_dir = Path('./.docs_cache')
raw_docs_pkl_path = docs_cache_dir / 'parsed_docs_cache.pkl'

docs = None

if os.path.exists(raw_docs_pkl_path):
  with open(raw_docs_pkl_path, 'rb') as f:
    docs = pickle.load(f)
else:
  docs = []
  for file in docs_dir.iterdir():
    docs.extend(PDFMinerLoader(file).load())

  with open(raw_docs_pkl_path, 'wb') as f:
    pickle.dump(docs, f)

## Split documents

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitted_docs_pkl_path = docs_cache_dir / 'splitted_docs_cache.pkl'

if os.path.exists(splitted_docs_pkl_path):
  with open(splitted_docs_pkl_path, 'rb') as f:
    splitted_docs = pickle.load(f)
else:
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=500,
    length_function=len,
    is_separator_regex=False,
  )
  splitted_docs = text_splitter.create_documents([doc.page_content for doc in docs])

  with open(splitted_docs_pkl_path, 'wb') as f:
    pickle.dump(splitted_docs, f)

len(splitted_docs)

17443

## Setup vector store

In [13]:
vector_store = Chroma.from_documents(
  documents=splitted_docs,
  embedding=cached_embeddings,
)
retriever = vector_store.as_retriever()

## Define JSON extractor

In [14]:
def extract_json(response):
  json_pattern = r'\{.*?\}'
  match = re.search(json_pattern, response, re.DOTALL)

  if match:
    return match.group().strip()

  return response

## Build LLM

In [15]:
llm = Ollama(model='llama3.1', temperature=0)

## Build chains

### Route chain

In [16]:
class RouteQuery(BaseModel):
  data_source: Literal['vectorstore', 'websearch'] = Field(
    description='Given a user question choose to route it to web search or a vectorstore.',
  )

route_parser = PydanticOutputParser(pydantic_object=RouteQuery)
route_retry_parser = RetryOutputParser.from_llm(
  parser=route_parser,
  llm=llm,
  max_retries=3,
)

route_template = """
You are an expert at routing a user question to a vectorstore or web search.
The vectorstore contains documents related to neurobiology and medicine.
Use the vectorstore for questions on these topics. For all else, use web-search.

{format_instructions}

{question}
"""
route_prompt = PromptTemplate(
  template=route_template,
  input_variables=['question'],
  partial_variables={'format_instructions': route_parser.get_format_instructions()},
)

question_router = RunnableParallel(
  completion=route_prompt | llm | extract_json, prompt_value=route_prompt
) | RunnableLambda(lambda x: route_retry_parser.parse_with_prompt(**x))
print(question_router.invoke({'question': 'Who will the Bears draft first in the NFL draft?'}))
print(question_router.invoke({'question': 'What is the order of the cranial nerves?'}))

data_source='websearch'
data_source='vectorstore'


### Grade documents chain

In [17]:
class GradeDocuments(BaseModel):
  binary_score: str = Field(description="Documents are relevant to the question, 'yes' or 'no'")

docs_grade_parser = PydanticOutputParser(pydantic_object=GradeDocuments)
docs_grade_retry_parser = RetryOutputParser.from_llm(
  parser=docs_grade_parser,
  llm=llm,
  max_retries=3,
)

docs_grade_template = """
You are a grader assessing relevance of a retrieved document to a user question.
If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant.
Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question.

{format_instructions}

User question:
{question}

Retrieved document:
{document}
"""
docs_grade_prompt = PromptTemplate(
  template=docs_grade_template,
  input_variables=['document', 'question'],
  partial_variables={'format_instructions': docs_grade_parser.get_format_instructions()},
)

docs_grade_grader = RunnableParallel(
  completion=docs_grade_prompt | llm | extract_json, prompt_value=docs_grade_prompt
) | RunnableLambda(lambda x: docs_grade_retry_parser.parse_with_prompt(**x))
docs_grade_grader.invoke({'question': 'What is the color of the sky?', 'document': 'The color of the sky is blue'})

GradeDocuments(binary_score='yes')

### Hallucinations chain

In [18]:
class GradeHallucinations(BaseModel):
  binary_score: str = Field(description="Answer is grounded in the facts, 'yes' or 'no'")

hallucination_parser = PydanticOutputParser(pydantic_object=GradeHallucinations)
hallucination_retry_parser = RetryOutputParser.from_llm(
  parser=hallucination_parser,
  llm=llm,
  max_retries=3,
)

hallucination_template = """
You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n
Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."

{format_instructions}

Set of facts:
{documents}

LLM generation:
{generation}
"""
hallucination_prompt = PromptTemplate(
  template=hallucination_template,
  input_variables=['question', 'generation'],
  partial_variables={'format_instructions': hallucination_parser.get_format_instructions()},
)

hallucination_grader = RunnableParallel(
  completion=hallucination_prompt | llm | extract_json, prompt_value=hallucination_prompt
) | RunnableLambda(lambda x: hallucination_retry_parser.parse_with_prompt(**x))
print(hallucination_grader.invoke({'documents': ['Sky is blue'], 'generation': 'The color of the sky is blue'}))

binary_score='yes'


### Answer grade chain

In [19]:
class GradeAnswer(BaseModel):
  binary_score: str = Field(description="Answer addresses the question, 'yes' or 'no'")

grade_parser = PydanticOutputParser(pydantic_object=GradeAnswer)
grade_retry_parser = RetryOutputParser.from_llm(
  parser=grade_parser,
  llm=llm,
  max_retries=3,
)

grade_template = """
You are a grader assessing whether an answer addresses / resolves a question. \n
Give a binary score 'yes' or 'no'. 'yes' means that the answer resolves the question.

User question:
{question}

LLM generation:
{generation}

{format_instructions}
"""
grade_prompt = PromptTemplate(
  template=grade_template,
  input_variables=['question', 'generation'],
  partial_variables={'format_instructions': grade_parser.get_format_instructions()},
)

answer_grader = RunnableParallel(
  completion=grade_prompt | llm | extract_json, prompt_value=grade_prompt
) | RunnableLambda(lambda x: grade_retry_parser.parse_with_prompt(**x))
print(answer_grader.invoke({"question": "What is the order of the cranial nerves?", 'generation': 'I do not know.'}))

binary_score='no'


## HyDE chain

In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

hyde_template = """
Please write a scientific paper passage to answer the question

Question: {question}

Passage:
"""
hyde_prompt = ChatPromptTemplate.from_template(hyde_template)
hyde_chain = hyde_prompt | llm | StrOutputParser()

hyde_chain.invoke({"question": 'What is the order of the cranial nerves ?'})

"Here's a scientific paper-style passage answering the question:\n\n**Title:** The Cranial Nerve Order: A Review and Classification\n\n**Abstract:**\nThe cranial nerves are a complex group of 12 pairs of nerves that arise from the brain and play crucial roles in various physiological functions, including sensory perception, motor control, and autonomic regulation. Despite their importance, the order of the cranial nerves has been a subject of debate among neuroscientists and anatomists for centuries. In this review, we provide an overview of the current classification system and discuss the historical context that led to the establishment of the traditional order.\n\n**Introduction:**\nThe cranial nerves are named according to their point of origin from the brainstem, with the first nerve arising from the forebrain (olfactory nerve) and the subsequent nerves emerging in a caudal direction. The traditional order of the cranial nerves is as follows:\n\nI. Olfactory nerve (CN I)\nII. Opti

In [21]:
prompt = hub.pull('rlm/rag-prompt')
rag_chain = prompt | llm | StrOutputParser()

rag_chain.invoke({"context": '', "question": 'What is the order of the cranial nerves ?'})

'The order of the 12 cranial nerves is as follows:\n\nI. Olfactory nerve\nII. Optic nerve\nIII. Occulomotor nerve\nIV. Trochlear nerve\nV. Trigeminal nerve\nVI. Abducens nerve\nVII. Facial nerve\nVIII. Auditory (or vestibulocochlear) nerve\nIX. Glossopharyngeal nerve\nX. Vagus nerve\nXI. Spinal accessory nerve\nXII. Hypoglossal nerve'

### Web search chain

In [22]:
from langchain_community.tools.tavily_search import TavilySearchResults
os.environ['TAVILY_API_KEY'] = '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'
web_search_tool = TavilySearchResults(k=5)

## Build graph app

In [23]:
from typing_extensions import TypedDict
from typing import List
from langchain.schema import Document

class GraphState(TypedDict):
  question: str
  generated_doc: str
  documents: List[str]
  web_search: str
  generation: str
  generations_num: int

def route_question(state):
  print('---ROUTE QUESTION---')

  question = state['question']

  source = question_router.invoke({'question': question})

  if source.data_source == 'websearch':
    print('---ROUTE QUESTION TO WEB SEARCH---')
    return 'websearch'
  elif source.data_source == 'vectorstore':
    print('---ROUTE QUESTION TO VECTOR STORE---')
    return 'vectorstore'

def generate_doc(state):
  print('---GENERATE DOCUMENT---')

  question = state['question']

  generated_doc = hyde_chain.invoke({'question': question})

  return {'question': question, 'generated_doc': generated_doc}

def retrieve(state):
  print('---RETRIEVE---')

  question = state['question']
  generated_doc = state['generated_doc']

  documents = retriever.invoke(generated_doc)

  return {'question': question, 'documents': documents}

def grade_documents(state):
  print('---CHECK DOCUMENT RELEVANCE TO QUESTION---')

  question = state['question']
  documents = state['documents']

  # Score each doc
  filtered_docs = []
  web_search = 'No'
  for index, d in enumerate(documents):
    print(f'---GRADE DOCUMENT ({index + 1}/{len(documents)})---')
    try:
      score = docs_grade_grader.invoke({'question': question, 'document': d.page_content})
      grade = score.binary_score
    except:
      grade = 'No'
    # Document relevant
    if grade.lower() == 'yes':
      print('---GRADE: DOCUMENT RELEVANT---')
      filtered_docs.append(d)
    # Document not relevant
    else:
      print('---GRADE: DOCUMENT NOT RELEVANT---')
      # We do not include the document in filtered_docs
      # We set a flag to indicate that we want to run web search
      web_search = 'Yes'
      continue

  print('final len', len(filtered_docs))

  return {
    'question': question,
    'documents': filtered_docs,
    'web_search': web_search,
  }

def decide_to_generate(state):
  print('---ASSESS GRADED DOCUMENTS---')

  web_search = state['web_search']

  if web_search == 'Yes':
    # All documents have been filtered check_relevance
    # We will re-generate a new query
    print('---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---')
    return 'websearch'
  else:
    # We have relevant documents, so generate answer
    print('---DECISION: GENERATE---')
    return 'generate'

def web_search(state):
  print('---WEB SEARCH---')

  question = state['question']
  documents = state.get('documents')

  try:
    docs = web_search_tool.invoke({'query': question})
    web_results = '\n'.join([d['content'] for d in docs])
    web_results = Document(page_content=web_results)
    if documents is not None:
      documents.append(web_results)
    else:
      documents = [web_results]
  except:
    pass

  return {
    'question': question,
    'documents': documents,
  }

def generate(state):
  print('---GENERATE---')

  question = state['question']
  documents = state['documents']
  generations_num = state.get('generations_num', 0)

  # RAG generation
  generation = rag_chain.invoke({'context': documents, 'question': question})
  return {
    'question': question,
    'documents': documents,
    'generation': generation,
    'generations_num': generations_num + 1,
  }

def grade_generation(state):
  print('---CHECK HALLUCINATIONS---')

  question = state['question']
  documents = state['documents']
  generation = state['generation']
  generations_num = state['generations_num']

  if generations_num >= 2:
    return 'useful'

  try:
    score = hallucination_grader.invoke({'documents': documents, 'generation': generation})
    grade = score.binary_score
  except:
    grade = 'no'

  # Check hallucination
  if grade == 'yes':
    print('---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---')
    # Check question-answering
    print('---GRADE GENERATION vs QUESTION---')
    try:
      score = answer_grader.invoke({'question': question,'generation': generation})
      grade = score.binary_score
    except:
      grade = 'no'

    if grade == 'yes':
      print('---DECISION: GENERATION ADDRESSES QUESTION---')
      return 'useful'
    else:
      print('---DECISION: GENERATION DOES NOT ADDRESS QUESTION---')
      return 'not useful'
  else:
    print('---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---')
    return 'not supported'

workflow = StateGraph(GraphState)
workflow.add_node('generate_doc', generate_doc)
workflow.add_node('retrieve', retrieve)
workflow.add_node('websearch', web_search)
workflow.add_node('generate', generate)
workflow.add_node('grade_documents', grade_documents)
workflow.set_conditional_entry_point(
  route_question,
  {
    'websearch': 'websearch',
    'vectorstore': 'generate_doc',
  },
)
workflow.add_edge('generate_doc', 'retrieve')
workflow.add_edge('retrieve', 'grade_documents')
workflow.add_conditional_edges(
  'grade_documents',
  decide_to_generate,
  {
    'websearch': 'websearch',
    'generate': 'generate',
  },
)
workflow.add_edge('websearch', 'generate')
workflow.add_conditional_edges(
  'generate',
  grade_generation,
  {
    'not supported': 'generate',
    'useful': END,
    'not useful': 'websearch',
  },
)

app = workflow.compile()

In [24]:
app.invoke({"question": 'What is the order of the cranial nerves?'})

---ROUTE QUESTION---
---ROUTE QUESTION TO VECTOR STORE---
---GENERATE DOCUMENT---
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE DOCUMENT (1/4)---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (2/4)---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (3/4)---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (4/4)---
---GRADE: DOCUMENT NOT RELEVANT---
final len 0
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---
---WEB SEARCH---
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---


{'question': 'What is the order of the cranial nerves?',
 'generated_doc': "Here's a scientific paper-style passage answering the question:\n\n**Title:** The Cranial Nerve Plexus: A Review of the Anatomical and Neurological Organization\n\n**Abstract:**\n\nThe cranial nerves, a complex network of 12 pairs of nerves that arise from the brainstem, play a crucial role in regulating various physiological functions. Understanding their organization is essential for appreciating the intricate relationships between the central nervous system and the peripheral nervous system. This review aims to provide an overview of the order of the cranial nerves, highlighting their anatomical and neurological characteristics.\n\n**Introduction:**\n\nThe cranial nerves are a group of nerves that emerge from the brainstem, which includes the midbrain, pons, and medulla oblongata. These nerves are responsible for controlling various functions such as sensation, movement, and autonomic regulation. The order o

## Evaluate RAG

In [25]:
qa_df = pd.read_csv('brainscape.csv')[:214]
qa_df

,question,answer
0,What are the afferent cranial nerve nuclei?,Trigeminal sensory nucleus- fibres carry gener...
1,What is the order of the cranial nerves ?,1-olfactory\n2-optic\n3-oculomotor\n4-trochlea...
2,What are the efferent cranial nerve nuclei?,Edinger-westphal nucleus\nOculomotor nucleus\n...
3,Which nuclei share the embryo logical origin -...,Oculomotor nucleus Trochlear nucleus Abducens ...
4,Which nuclei share the embryo logical origin- ...,Trigeminal motor nucleus Facial motor nucleus ...
...,...,...
209,What are spontaneous active bursting neurones ?,These are pacemaker neurons They don’t require...
210,What is the patch-clamp technique ?,Enables recordings of electrical activity of a...
211,What toxin can be used to block sodium currents ?,Tetrodotoxin-it blocks sodium channels
212,What toxin can block potassium currents ?,Tetrammonium - blocks potassium channels


In [26]:
questions = list(qa_df['question'].tolist())
expected_answers = list(qa_df['answer'].tolist())
predicted_answers = []

for index, question in tqdm(enumerate(questions)):
  answer = app.invoke({'question': question})['generation']
  predicted_answers.append(answer)

0it [00:00, ?it/s]

---ROUTE QUESTION---
---ROUTE QUESTION TO VECTOR STORE---
---GENERATE DOCUMENT---
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE DOCUMENT (1/4)---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (2/4)---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (3/4)---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (4/4)---
---GRADE: DOCUMENT NOT RELEVANT---
final len 0
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---
---WEB SEARCH---
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---


1it [00:14, 14.31s/it]

---DECISION: GENERATION ADDRESSES QUESTION---
---ROUTE QUESTION---
---ROUTE QUESTION TO VECTOR STORE---
---GENERATE DOCUMENT---
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE DOCUMENT (1/4)---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (2/4)---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (3/4)---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (4/4)---
---GRADE: DOCUMENT NOT RELEVANT---
final len 0
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---
---WEB SEARCH---
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---


2it [00:25, 12.26s/it]

...

---DECISION: GENERATION ADDRESSES QUESTION---
---ROUTE QUESTION---
---ROUTE QUESTION TO VECTOR STORE---
---GENERATE DOCUMENT---
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE DOCUMENT (1/4)---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (2/4)---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (3/4)---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (4/4)---
---GRADE: DOCUMENT NOT RELEVANT---
final len 0
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---
---WEB SEARCH---
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---


213it [43:51, 10.17s/it]

---DECISION: GENERATION ADDRESSES QUESTION---
---ROUTE QUESTION---
---ROUTE QUESTION TO VECTOR STORE---
---GENERATE DOCUMENT---
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE DOCUMENT (1/4)---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (2/4)---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (3/4)---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (4/4)---
---GRADE: DOCUMENT NOT RELEVANT---
final len 0
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---
---WEB SEARCH---
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION DOES NOT ADDRESS QUESTION---
---WEB SEARCH---
---GENERATE---


214it [44:10, 12.38s/it]

---CHECK HALLUCINATIONS---


In [27]:
cos_score = embeddings_cosine_sim_metric(expected_answers, predicted_answers)
bleu_score = bleu_metric(expected_answers, predicted_answers)
rogue_1_score = rogue_1_metric(expected_answers, predicted_answers)
rogue_l_score = rogue_l_metric(expected_answers, predicted_answers)

cos_score, bleu_score, rogue_1_score, rogue_l_score

(0.6234251400832006,
 0.018905001222402712,
 0.2127606292570124,
 0.17643369262342895)